# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20

query = {'scores.Hygiene': 20}
# Use count_documents to display the number of documents in the result
print("Number of documents in result: ", establishments.count_documents(query))
results = establishments.find(query)
# Display the first document in the results using pprint
pprint(results[0])

Number of documents in result:  2
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650d0bda6962a188f2281a3f'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': 'decimal',
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 

In [7]:
import pandas as pd
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(len(results_df))

2


In [8]:
# Display the first 10 rows of the DataFrame
results_df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,latitude,longitude
0,650d0bda6962a188f2281a3f,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI...",FHRS,"{'longitude': '0.27694', 'latitude': '50.769705'}",,4613.888288,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",decimal,decimal
1,650d0bda6962a188f2281dc0,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceI...",FHRS,"{'longitude': '1.278721', 'latitude': '51.8575...",,4617.965824,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",decimal,decimal


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [9]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'RatingValue': {'$gte': 4}}
# Use count_documents to display the number of documents in the result
print("Number of documents in result: ", establishments.count_documents(query))
result = establishments.find(query)
# Display the first document in the results using pprint
pprint(result[0])

Number of documents in result:  7152
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650d0bd86962a188f227ff0d'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'latitude': 'decimal',
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043

In [10]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(result)
# Display the number of rows in the DataFrame
print(len(result_df))


7152


"### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant Penang Flavours"?

In [11]:
# Create the pipeline: match, then sort, then limit to 5
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {'geocode.latitude': {'$gte':latitude-degree_search, '$lte':latitude+degree_search}, 
         'geocode.longitude': {'$gte': longitude-degree_search, '$lte': longitude+degree_search}, 
         'RatingValue':5}
sort = [('scores.Hygiene', 1)]
limit = 5

# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))

[]


In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# #degree_search = 0.01
# latitude = 51.49014200
# longitude = 0.08384000



# query = {'geocode.latitude': {'$gte':latitude-degree_search, '$lte':latitude+degree_search},
#          'geocode.longitude': {'$gte': longitude-degree_search, '$lte': longitude+degree_search}, 'Rating Value': 5}
  
# sort =  [('scores.Hygiene',1)]


# # Print the results
# results = establishments.find(query).sort(sort)
# pprint(results)



In [13]:
# Convert result to Pandas DataFrame
results_df = pd.DataFrame(results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match':{'scores.hygiene': 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group':{'_id':'LocalAuthority','count': { '$sum': 1 }} }
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort':{'count': -1}}
# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
# Print the first 10 results
results = list(establishments.aggregate(pipeline))
pprint(results)


[]


In [18]:
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
result_df.count()
# Display the first 10 rows of the DataFrame
pprint(results[0:10])


[]
